# ORM Usage
This notebook will look into integrating the Turing SDK by implementing the `ShortAnswerQuestion` object into an ORM model. For the purpose of simplicity, we will use the Django ORM in this notebook. However, as you will see, this integration works regardless of your ORM choice.

To grade questions with Turing, you need to implement the two stages of the question grading lifecycle:
1. **Question object creation**
2. **Question grading request**

Every request for question grading must address these two steps in one way or another. First, let's worry about creating a question object. As the name of this notebook suggests, we are going to create questions via an ORM model. To do this, we can implement an method on the ORM model, which utitlizes the fields of the model to create a `ShortAnswerQuestion` object.

First let's set the stage with an ORM model.

## Defining an ORM Model

In [ ]:
# models.py

from django.db import models

class QuizQuestion(models.Model):
    """Model representing a question in a quiz."""

    # Choices for `queston_type` field
    FACTUAL = "FA"
    OPTION = "OP"
    ANALYSIS = "AN"
    QUESTION_TYPE_CHOICES = [
        (FACTUAL, "Factual"),
        (OPINION, "Opinion"),
        (ANALYSIS, "Analysis")
    ]

    # Model Fields
    _id = models.UUIDField(primary_key=True)
    question = models.CharField(max_length=100)
    example_answer = models.CharField(max_length=250)
    question_type = models.CharField(max_length=2, choices =QUESTION_TYPE_CHOICES, default=FACUTAL)

    def __str__(self):
        """String for representing the QuizQuestion object."""
        return self.question

With our model defined we can move on to handle creating the Turing `ShortAnswerQuestion` objects directly through the model.

## Creating `ShortAnswerQuestion` Objects

In [ ]:
from django.db import models

from turing.question import ShortAnswerQuestion, RubricType

class QuizQuestion(models.Model):
    FACTUAL = "FA"
    OPTION = "OP"
    ANALYSIS = "AN"
    QUESTION_TYPE_CHOICES = [
        (FACTUAL, "Factual"),
        (OPINION, "Opinion"),
        (ANALYSIS, "Analysis")
    ]

    _id = models.UUIDField(primary_key=True)
    question = models.CharField(max_length=100)
    example_answer = models.CharField(max_length=250)
    question_type = models.CharField(max_length=2, choices =QUESTION_TYPE_CHOICES, default='FACTUAL')

    def __str__(self):
        return self.question

    @property
    def rubric_type(self):
        """Converts the question_type field to a RubricType object."""
        match self.question_type:
            case self.FACTUAL:
                return RubricType.FACTUAL
            case self.OPINION:
                return RubricType.COMMUNICATION_RUBRIC
            case self.ANALYSIS:
                return RubricType.ANALYTICAL_RUBRIC
            case _:
                return RubricType.CUSTOM_RUBRIC

    @property
    def turing_question(self):
        """Converts the QuizQuestion to a Turing ShortAnswerQuestion"""
        return ShortAnswerQuestion.from_rubric_type(
            question=self.question,
            example_answer=self.example_answer,
            rubric_type=self.rubric_type
        )

By implementing two simple properties into our ORM model, we were able to convert our standard Django model into a `ShortAnswerQuestion` factory. Now, we can finish the implementation by defining a `grade` method on our question model.

## Grading Method
To keep with the Django theme, let's imagine we are in a view function now. We will call this the question grading view. In this view, we will query for a question through the `QuizQuestion` model, then use our properites and the provided `grade` method of the `ShortAnswerQuestion` object to grade the student's response with Turing. 

In [ ]:

from typing import Tuple

from django.db import models
from turing.question import ShortAnswerQuestion, RubricType

class QuizQuestion(models.Model):
    FACTUAL = "FA"
    OPTION = "OP"
    ANALYSIS = "AN"
    QUESTION_TYPE_CHOICES = [
        (FACTUAL, "Factual"),
        (OPINION, "Opinion"),
        (ANALYSIS, "Analysis")
    ]

    _id = models.UUIDField(primary_key=True)
    question = models.CharField(max_length=100)
    example_answer = models.CharField(max_length=250)
    question_type = models.CharField(max_length=2, choices =QUESTION_TYPE_CHOICES, default='FACTUAL')

    def __str__(self):
        return self.question

    @property
    def rubric_type(self):
        """Converts the question_type field to a RubricType object."""
        match self.question_type:
            case self.FACTUAL:
                return RubricType.FACTUAL
            case self.OPINION:
                return RubricType.COMMUNICATION_RUBRIC
            case self.ANALYSIS:
                return RubricType.ANALYTICAL_RUBRIC
            case _:
                return RubricType.CUSTOM_RUBRIC

    @property
    def turing_question(self):
        """Converts the QuizQuestion to a Turing ShortAnswerQuestion"""
        return ShortAnswerQuestion.from_rubric_type(
            question=self.question,
            example_answer=self.example_answer,
            rubric_type=self.rubric_type
        )

    def grade(self, answer: str) -> Tuple[str, float]:
        """Grades the answer and returns the feedback and score."""
        feedback, score = self.turing_question.grade(answer)
        return feedback, score

Now we can access our model object throughout the app. To finish the demonstration, let's look at how this model method could be invoked from a view.
## Grading Questions
We can implement the `grade` method of the question into a view function, to make Turing's grading fucntionality available to your servers clients. 

In [ ]:
from django.http import HttpRequest, HttpResponse
from django.views.decorators import require_http_methods

@require_http_methods(["POST"])
def grade_short_answer_view(request: HttpRequest, question_id: str) -> HttpResponse:
    """View for grading a short answer question."""
    # Retrieve the question model from the database
    question = Question.objects.get(_id=question_id)

    # Extract the student's answer from the request payload
    students_answer = request.POST.get("answer")

    # Grade the answer
    feedback, score = question.grade(students_answer)

    # Return the feedback and score to the student
    return HttpResponse({'feedback': feedback, 'score': score}, status=200)


From this standpoint, it is clear to see how ORM integration is the recommended installation pattern for Turing's SDK. This implementation gives us a clean interface for utilizing Turing's API as an instance method on our projects predfined models. This gives implementation provides a setup in which the grading logic is completely contained to the model-layer of your application. 

It is important to note that your use case may vary. You may find it useful to subclass the `ShortAnswerQuestion` object either directly through your ORM model or to build a wrapper for the the question object that can enable Turing to interact with multiple components of your application. The main purpose of this notebook was to demonstrate how simple and flexible the Turing SDK can be. 